In [1]:
import pandas as pd
import meterdatalogic as ml

TZ = "Australia/Brisbane"


In [2]:
# 7 days of half-hourly data for one NMI
rng = pd.date_range("2025-01-01", periods=48*7, freq="30min", tz=TZ)
df = pd.DataFrame({
    "t_start": rng,
    "nmi": "Q1234567890",
    "channel": "E1",
    "kwh": 0.5
})
df.head()


,t_start,nmi,channel,kwh
0,2025-01-01 00:00:00+10:00,Q1234567890,E1,0.5
1,2025-01-01 00:30:00+10:00,Q1234567890,E1,0.5
2,2025-01-01 01:00:00+10:00,Q1234567890,E1,0.5
3,2025-01-01 01:30:00+10:00,Q1234567890,E1,0.5
4,2025-01-01 02:00:00+10:00,Q1234567890,E1,0.5


In [7]:
canon_df = ml.ingest.from_nem12("../data/Sample1.csv", tz="Australia/Brisbane")
ml.validate.assert_canon(canon_df)

print(canon_df.index.tz, df.index.name)
print(canon_df.head())
print(canon_df["flow"].value_counts())
canon_df

Australia/Brisbane None
                               nmi channel         flow    kwh  cadence_min
t_start                                                                    
2014-05-06 00:00:00+10:00  SAMPLE1      E1  grid_import  0.144           30
2014-05-06 00:30:00+10:00  SAMPLE1      E1  grid_import  0.163           30
2014-05-06 01:00:00+10:00  SAMPLE1      E1  grid_import  0.106           30
2014-05-06 01:30:00+10:00  SAMPLE1      E1  grid_import  0.144           30
2014-05-06 02:00:00+10:00  SAMPLE1      E1  grid_import  0.169           30
flow
grid_import    35136
Name: count, dtype: int64


,nmi,channel,flow,kwh,cadence_min
t_start,,,,,
2014-05-06 00:00:00+10:00,SAMPLE1,E1,grid_import,0.144,30
2014-05-06 00:30:00+10:00,SAMPLE1,E1,grid_import,0.163,30
2014-05-06 01:00:00+10:00,SAMPLE1,E1,grid_import,0.106,30
2014-05-06 01:30:00+10:00,SAMPLE1,E1,grid_import,0.144,30
2014-05-06 02:00:00+10:00,SAMPLE1,E1,grid_import,0.169,30
...,...,...,...,...,...
2016-05-06 21:30:00+10:00,SAMPLE1,E1,grid_import,0.394,30
2016-05-06 22:00:00+10:00,SAMPLE1,E1,grid_import,0.356,30
2016-05-06 22:30:00+10:00,SAMPLE1,E1,grid_import,0.369,30


In [4]:
# canon_df = ml.ingest.from_dataframe(df, tz=TZ)
# ml.validate.assert_canon(canon_df)

# print("✅ Canonicalized columns:", canon_df.columns.tolist())
# canon_df.head(3)


In [8]:
daily = ml.transform.groupby_day(canon_df)
daily.head()


flow,grid_import
day,
2014-05-06 00:00:00+10:00,25.440
2014-05-07 00:00:00+10:00,25.228
2014-05-08 00:00:00+10:00,28.367
2014-05-09 00:00:00+10:00,23.671
2014-05-10 00:00:00+10:00,29.284


In [9]:
monthly = ml.transform.groupby_month(canon_df)
monthly.head()


flow,month,grid_import
0,2014-05,731.292
1,2014-06,871.503
2,2014-07,907.499
3,2014-08,708.981
4,2014-09,636.617


In [10]:
profile24 = ml.transform.profile24(canon_df)
profile24.head()


flow,slot,grid_import
0,00:00,0.228359
1,00:30,0.208526
2,01:00,0.186454
3,01:30,0.180960
4,02:00,0.189959


In [11]:
bands = [
    {"name": "off", "start": "00:00", "end": "16:00"},
    {"name": "peak", "start": "16:00", "end": "21:00"},
    {"name": "shoulder", "start": "21:00", "end": "24:00"},
]
tou = ml.transform.tou_bins(canon_df, bands)
tou.head()


band,month,off,peak,shoulder
0,2014-05,371.368,234.112,125.812
1,2014-06,449.213,266.161,156.129
2,2014-07,448.150,275.345,184.004
3,2014-08,413.570,190.108,105.303
4,2014-09,373.269,173.923,89.425


In [12]:
demand = ml.transform.demand_window(canon_df, start="16:00", end="21:00", days="MF")
demand.head()


,month,demand_kw
0,2014-05,5.276
1,2014-06,7.500
2,2014-07,5.638
3,2014-08,4.462
4,2014-09,3.888


In [13]:
summary = ml.summary.summarize(canon_df)
summary["meta"], summary["energy"], summary["per_day_avg_kwh"]


({'nmis': 1,
  'start': '2014-05-06T00:00:00+10:00',
  'end': '2016-05-06T23:30:00+10:00',
  'cadence_min': 30,
  'days': 732},
 {'grid_import': 17410.424},
 23.784732240437158)

In [14]:
import json
print(json.dumps(summary, indent=2)[:800])


{
  "meta": {
    "nmis": 1,
    "start": "2014-05-06T00:00:00+10:00",
    "end": "2016-05-06T23:30:00+10:00",
    "cadence_min": 30,
    "days": 732
  },
  "energy": {
    "grid_import": 17410.424
  },
  "per_day_avg_kwh": 23.784732240437158,
  "peaks": {
    "max_interval_kwh": 3.862,
    "max_interval_time": "2016-04-18 20:30:00+10:00"
  },
  "profile24": [
    {
      "slot": "00:00",
      "grid_import": 0.22835928961748633
    },
    {
      "slot": "00:30",
      "grid_import": 0.208525956284153
    },
    {
      "slot": "01:00",
      "grid_import": 0.18645355191256832
    },
    {
      "slot": "01:30",
      "grid_import": 0.1809603825136612
    },
    {
      "slot": "02:00",
      "grid_import": 0.18995901639344265
    },
    {
      "slot": "02:30",
      "grid_import": 0.200


In [15]:
# Define a simple ToU plan
plan = ml.types.Plan(
    usage_bands=[
        ml.types.ToUBand("off", "00:00", "16:00", 22.0),
        ml.types.ToUBand("peak", "16:00", "21:00", 45.0),
        ml.types.ToUBand("shoulder", "21:00", "24:00", 28.0),
    ],
    demand=ml.types.DemandCharge("16:00", "21:00", "MF", 12.0),
    fixed_c_per_day=95.0,
    feed_in_c_per_kwh=6.0
)

monthly_cost = ml.pricing.estimate_monthly_cost(canon_df, plan)
monthly_cost.head()


,month,energy_cost,demand_cost,fixed_cost,feed_in_credit,total
0,2014-05,222.27872,63.312,28.915625,-0.0,314.506345
1,2014-06,262.31543,90.000,28.915625,-0.0,381.231055
2,2014-07,274.01937,67.656,28.915625,-0.0,370.590995
3,2014-08,206.01884,53.544,28.915625,-0.0,288.478465
4,2014-09,185.42353,46.656,28.915625,-0.0,260.995155


In [16]:
payload = {
    "summary": ml.summary.summarize(canon_df),
    "daily": ml.transform.groupby_day(canon_df).reset_index().to_dict("records"),
    "monthly": ml.transform.groupby_month(canon_df).to_dict("records"),
    "profile24": ml.transform.profile24(canon_df).to_dict("records"),
    "tou": ml.transform.tou_bins(canon_df, bands).to_dict("records"),
    "cost": ml.pricing.estimate_monthly_cost(canon_df, plan).to_dict("records")
}

# Quick preview
import json
print(json.dumps(payload["summary"], indent=2)[:800])


{
  "meta": {
    "nmis": 1,
    "start": "2014-05-06T00:00:00+10:00",
    "end": "2016-05-06T23:30:00+10:00",
    "cadence_min": 30,
    "days": 732
  },
  "energy": {
    "grid_import": 17410.424
  },
  "per_day_avg_kwh": 23.784732240437158,
  "peaks": {
    "max_interval_kwh": 3.862,
    "max_interval_time": "2016-04-18 20:30:00+10:00"
  },
  "profile24": [
    {
      "slot": "00:00",
      "grid_import": 0.22835928961748633
    },
    {
      "slot": "00:30",
      "grid_import": 0.208525956284153
    },
    {
      "slot": "01:00",
      "grid_import": 0.18645355191256832
    },
    {
      "slot": "01:30",
      "grid_import": 0.1809603825136612
    },
    {
      "slot": "02:00",
      "grid_import": 0.18995901639344265
    },
    {
      "slot": "02:30",
      "grid_import": 0.200
